In [0]:
import pandas as pd
import requests
import json
from pprint import pprint
import time
# from config import yelp_api_key

In [0]:
yelp_api_key = ""

In [0]:
url = "https://api.yelp.com/v3/businesses/search"

In [0]:
zip_codes=pd.read_csv('LA_Zips.csv')
zip_codes.head()





,ZIP Code
0,90001
1,90002
2,90003
3,90004
4,90005


In [0]:
headers = {'Authorization': 'Bearer %s' % yelp_api_key,}

In [0]:
total_businesses = []

for Z in zip_codes['ZIP Code']:
    print(Z)
    url_params = {'categories':'restaurants', 'location': Z, 'is_closed': 'False','offset': 0, 'limit': 50,}
    response = requests.request('GET', url, headers=headers, params=url_params)

    current_offset=0
    current_limit=50
    
    total_response = response.json()['total']

    while current_offset <= total_response:
        print(current_offset)

        url_params['offset']=current_offset
        response = requests.request('GET', url, headers=headers, params=url_params)
        if 'businesses' in response.json():
            total_businesses += response.json()['businesses']
            
        current_offset += current_limit
#         pprint(response.json())
#     print(len(total_businesses))
    

In [0]:
list = []

for biz in total_businesses:
    
    id_ = biz['id']
    name = biz['name']
    phone = biz['phone']
    addresses = biz['location']['display_address']
    address = '  '.join(addresses)
    
    services = biz['categories']
    
    alias = ''
    title = ''
    
    for service in services:
        alias += service['alias'] + ', '
        title += service['title'] + ', '
        
    # more verbose alternate
    #for service in services
#         a_alias = service['alias']
#         alias += a_alias
        
    
    rating = biz['rating']
    review_count = biz['review_count']
    lat = biz['coordinates']['latitude']
    lon = biz['coordinates']['longitude']
    
    if lat is None:
        continue
    if lon is None:
        continue

    yelp_dict = {
                    'id': id_,
                    'name': name,
                    'phone': phone,
                    'address' : address,
                    'category_alias': alias,
                    'category_title': title,
                    'rating' : rating,
                    'review_count' : review_count,
                    'lat': lat,
                    'lon': lon,
                        }



    pprint(yelp_dict)

    list.append(yelp_dict)
#     print(biz['coordinates'])



    
#     print(name  + '  ' +  phone + '  ' + review_count  + '  ' + lat   + '  ' + lon    + '\n' +
#           '  ' +  address  + '  ' +  rating) 

In [0]:
len(total_businesses)

139390

In [0]:
len(list)

1000

In [0]:
LA_Rest = pd.DataFrame(list)
# LA_Rest.dropna(axis=0, how='any',)

# LA_Rest.head()

In [0]:
len(LA_Rest)

1000

In [0]:
# saving the dataframe 
LA_Rest.to_csv('LA_rest_yelp.csv') 